In [1]:
import glob
import numpy as np
import os
import pandas as pd
import pickle
import prefect
import subprocess
import tempfile
import wolf

Error configuring prefect logger


In [20]:
import importlib
importlib.reload(wolf)

<module 'wolf' from '/home/opriebe/anaconda3/envs/hapaseg/lib/python3.8/site-packages/wolf/__init__.py'>

In [21]:
# for Hapaseg itself
hapaseg = wolf.ImportTask(
  task_path = "../", # TODO: make remote
  task_name = "hapaseg"
)

In [3]:
# localize reference files to RODISK
localization_task = wolf.LocalizeToDisk(
  files = dict(
    ref_fasta = "gs://getzlab-workflows-reference_files-oa/hg19/Homo_sapiens_assembly19.fasta",
    ref_fasta_idx = "gs://getzlab-workflows-reference_files-oa/hg19/Homo_sapiens_assembly19.fasta.fai",
    ref_fasta_dict = "gs://getzlab-workflows-reference_files-oa/hg19/Homo_sapiens_assembly19.dict",
    coverage_csv='/home/opriebe/dev/HapASeg/exome/6_C1D1_META.cov',
    allelic_clusters_object='/home/opriebe/dev/HapASeg/exome/6_C1D1_META.DP_clusts.auto_ref_correct.overdispersion92.no_phase_correct.npz',
    SNPs_pickle='/home/opriebe/dev/HapASeg/exome/6_C1D1_META.SNPs.pickle',
    covariate_dir='/home/opriebe/dev/HapASeg/covars',
  )
)

In [4]:
loc_res = localization_task.run()

[2022-02-17 20:45:03+0000] WARNING - prefect | Available disk storage at /mnt/nfs is small (68 GB remaining)
[2022-02-17 20:45:03+0000] INFO - prefect | Starting Slurm controller ...
[2022-02-17 20:45:03+0000] INFO - prefect | Waiting up to 60 seconds for Slurm controller to start ...
[2022-02-17 20:45:17+0000] INFO - prefect | Hashing file /home/opriebe/dev/HapASeg/covars/GSE137764_H1.hg19_raw_liftover.pickle (0 MiB)
[2022-02-17 20:45:17+0000] INFO - prefect | Hashing file /home/opriebe/dev/HapASeg/covars/GC.pickle (0 MiB)
[2022-02-17 20:45:17+0000] INFO - prefect | Hashing file /home/opriebe/dev/HapASeg/covars/GSE137764_H1_GaussiansGSE137764_mooth_scaled_autosome.mat (0 MiB)
[2022-02-17 20:45:17+0000] INFO - prefect | Hashing file /home/opriebe/dev/HapASeg/covars/GSE137764_H1.hg38.pickle (0 MiB)
[2022-02-17 20:45:17+0000] INFO - prefect | Hashing file /home/opriebe/dev/HapASeg/covars/GSE137764_H1.hg19_liftover.pickle (0 MiB)
[2022-02-17 20:45:17+0000] WARNING - prefect | Job avoidanc

Listed 0 items.


[2022-02-17 20:48:25+0000] INFO - prefect | Finished with status COMPLETED


In [5]:
loc_res

{'SNPs_pickle': 'rodisk://canine-5eabafd987978c5006fcb69a64dd7a4e/SNPs_pickle/6_C1D1_META.SNPs.pickle',
 'allelic_clusters_object': 'rodisk://canine-5eabafd987978c5006fcb69a64dd7a4e/allelic_clusters_object/6_C1D1_META.DP_clusts.auto_ref_correct.overdispersion92.no_phase_correct.npz',
 'covariate_dir': 'rodisk://canine-5eabafd987978c5006fcb69a64dd7a4e/covariate_dir/covars',
 'coverage_csv': 'rodisk://canine-5eabafd987978c5006fcb69a64dd7a4e/coverage_csv/6_C1D1_META.cov',
 'ref_fasta': 'rodisk://canine-5eabafd987978c5006fcb69a64dd7a4e/ref_fasta/Homo_sapiens_assembly19.fasta',
 'ref_fasta_dict': 'rodisk://canine-5eabafd987978c5006fcb69a64dd7a4e/ref_fasta_dict/Homo_sapiens_assembly19.dict',
 'ref_fasta_idx': 'rodisk://canine-5eabafd987978c5006fcb69a64dd7a4e/ref_fasta_idx/Homo_sapiens_assembly19.fasta.fai'}

In [25]:
np.arange(20)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [6]:
clusts = [i for i in range(20)]

In [7]:
cov_mcmc_task = hapaseg.Hapaseg_coverage_mcmc(
inputs={
        "coverage_csv":loc_res["coverage_csv"],
        "allelic_clusters_object":loc_res["allelic_clusters_object"],
        "SNPs_pickle":loc_res["SNPs_pickle"],
        "covariate_dir":loc_res["covariate_dir"],
        "num_draws":10,
        "cluster_num":clusts,
        "allelic_sample":499,
        "ref_file_path":loc_res["ref_fasta"]}
)
cov_mcmc_results = cov_mcmc_task.run()

[2022-02-17 20:48:25+0000] WARNING - prefect | Available disk storage at /mnt/nfs is small (68 GB remaining)
[2022-02-17 20:48:25+0000] INFO - prefect | Starting Slurm controller ...
[2022-02-17 20:48:25+0000] INFO - prefect | Waiting up to 60 seconds for Slurm controller to start ...
[2022-02-17 20:48:26+0000] WARNING - prefect | Cannot recover preexisting task outputs: [Errno 2] No such file or directory: '/mnt/nfs/workspace/Hapaseg_coverage_mcmc__2022-02-17--20-48-26_1dspx3a_p3h1szq_d0grvvdwbc3ym/jobs/0'
[2022-02-17 20:48:26+0000] WARNING - prefect | Overwriting output and aborting job avoidance.
[2022-02-17 20:48:26+0000] INFO - prefect | Localizing inputs...
[2022-02-17 20:48:26+0000] INFO - prefect | Job staged on SLURM controller in: /mnt/nfs/workspace/Hapaseg_coverage_mcmc__2022-02-17--20-48-26_1dspx3a_p3h1szq_d0grvvdwbc3ym
[2022-02-17 20:48:26+0000] INFO - prefect | Preparing pipeline script
[2022-02-17 21:02:04+0000] INFO - prefect | Finished with statuses COMPLETED: 20


In [8]:
cov_mcmc_gather = hapaseg.Hapaseg_collect_coverage_mcmc(
    inputs={
        "cov_mcmc_files":[cov_mcmc_results["cov_segmentation_data"]],
        "cov_df_pickle":cov_mcmc_results["cov_df_segmentation"][0]
    }
)

In [9]:
gather_results = cov_mcmc_gather.run()

[2022-02-17 21:02:04+0000] WARNING - prefect | Available disk storage at /mnt/nfs is small (68 GB remaining)
[2022-02-17 21:02:04+0000] INFO - prefect | Starting Slurm controller ...
[2022-02-17 21:02:04+0000] INFO - prefect | Waiting up to 60 seconds for Slurm controller to start ...
[2022-02-17 21:02:05+0000] WARNING - prefect | Cannot recover preexisting task outputs: [Errno 2] No such file or directory: '/mnt/nfs/workspace/Hapaseg_collect_coverage_mcmc__2022-02-17--21-02-05_fueiipq_p3h1szq_scld2nyne5ct0/jobs/0'
[2022-02-17 21:02:05+0000] WARNING - prefect | Overwriting output and aborting job avoidance.
[2022-02-17 21:02:05+0000] INFO - prefect | Localizing inputs...
[2022-02-17 21:02:05+0000] INFO - prefect | Job staged on SLURM controller in: /mnt/nfs/workspace/Hapaseg_collect_coverage_mcmc__2022-02-17--21-02-05_fueiipq_p3h1szq_scld2nyne5ct0
[2022-02-17 21:02:05+0000] INFO - prefect | Preparing pipeline script
[2022-02-17 21:02:36+0000] INFO - prefect | Finished with status COMPL

In [10]:
gather_results

<WolfTaskResults: State: COMPLETED
                  Inputs: {'cov_df_pickle': '/mnt/nfs/workspace/Hapaseg_coverage_mcmc__2022-02-17--20-48-26_1dspx3a_p3h1szq_d0grvvdwbc3ym/outputs/0/cov_df_segmentation/coverage_mcmc_clusters/cov_df.pickle', 'cov_mcmc_files': '[<canine.localization.file_handlers.HandleRegularFile object at 0x7f8d7e16fd60>, <canine.localization.file_handlers.HandleRegularFile object at 0x7f8d7e16fac0>, <canine.localization.file_handlers.HandleRegularFile object at 0x7f8d7e1c7b50>, <canine.localization.file_handlers.HandleRegularFile object at 0x7f8d7e1c7460>, <canine.localization.file_handlers.HandleRegularFile object at 0x7f8d84241e20>, <canine.localization.file_handlers.HandleRegularFile object at 0x7f8d84241c40>, <canine.localization.file_handlers.HandleRegularFile object at 0x7f8d84241df0>, <canine.localization.file_handlers.HandleRegularFile object at 0x7f8d84241340>, <canine.localization.file_handlers.HandleRegularFile object at 0x7f8d84241dc0>, <canine.localizati

In [11]:
cov_dp_task = hapaseg.Hapaseg_coverage_dp(
    inputs={
        "f_cov_df":cov_mcmc_results["cov_df_segmentation"][0],
        "cov_mcmc_data": gather_results["cov_collected_data"],
        "num_segmentation_samples":10,
        "num_draws":10
    }
)

In [ ]:
cov_dp_results = cov_dp_task.run()

[2022-02-17 21:02:36+0000] WARNING - prefect | Available disk storage at /mnt/nfs is small (68 GB remaining)
[2022-02-17 21:02:36+0000] INFO - prefect | Starting Slurm controller ...
[2022-02-17 21:02:36+0000] INFO - prefect | Waiting up to 60 seconds for Slurm controller to start ...
[2022-02-17 21:02:37+0000] WARNING - prefect | Cannot recover preexisting task outputs: [Errno 2] No such file or directory: '/mnt/nfs/workspace/Hapaseg_coverage_dp__2022-02-17--21-02-37_vr5yroi_p3h1szq_zffqaw5z5n03c/jobs/0'
[2022-02-17 21:02:37+0000] WARNING - prefect | Overwriting output and aborting job avoidance.
[2022-02-17 21:02:37+0000] INFO - prefect | Localizing inputs...
[2022-02-17 21:02:37+0000] INFO - prefect | Job staged on SLURM controller in: /mnt/nfs/workspace/Hapaseg_coverage_dp__2022-02-17--21-02-37_vr5yroi_p3h1szq_zffqaw5z5n03c
[2022-02-17 21:02:37+0000] INFO - prefect | Preparing pipeline script


In [15]:
cov_dp_results["cov_dp_object"]

'/mnt/nfs/workspace/Hapaseg_coverage_dp__2022-02-17--21-02-37_vr5yroi_p3h1szq_zffqaw5z5n03c/outputs/0/cov_dp_object/Cov_DP_model.pickle'

In [16]:
cov_dp_results["cov_dp_figure"]

'/mnt/nfs/workspace/Hapaseg_coverage_dp__2022-02-17--21-02-37_vr5yroi_p3h1szq_zffqaw5z5n03c/outputs/0/cov_dp_figure/coverage_figures/coverage_draw_9.png'

In [23]:
gen_acdp_task = hapaseg.Hapaseg_acdp_generate_df(
    inputs = {
        "SNPs_pickle":loc_res["SNPs_pickle"],
        "allelic_clusters_object":loc_res["allelic_clusters_object"],
        "coverage_dp_object":cov_dp_results["cov_dp_object"],
        "allelic_draw_index":499,
        "ref_file_path":loc_res["ref_fasta"]
    }
)

In [24]:
gen_acdp_results = gen_acdp_task.run()

[2022-02-18 13:40:36+0000] WARNING - prefect | Available disk storage at /mnt/nfs is small (68 GB remaining)
[2022-02-18 13:40:36+0000] INFO - prefect | Starting Slurm controller ...
[2022-02-18 13:40:36+0000] INFO - prefect | Waiting up to 60 seconds for Slurm controller to start ...
[2022-02-18 13:40:37+0000] WARNING - prefect | Cannot recover preexisting task outputs: [Errno 2] No such file or directory: '/mnt/nfs/workspace/Hapaseg_acdp_generate_df__2022-02-18--13-40-37_b3hxleq_p3h1szq_i244yc4nxd4iq/jobs/0'
[2022-02-18 13:40:37+0000] WARNING - prefect | Overwriting output and aborting job avoidance.
[2022-02-18 13:40:37+0000] INFO - prefect | Localizing inputs...
[2022-02-18 13:40:37+0000] INFO - prefect | Job staged on SLURM controller in: /mnt/nfs/workspace/Hapaseg_acdp_generate_df__2022-02-18--13-40-37_b3hxleq_p3h1szq_i244yc4nxd4iq
[2022-02-18 13:40:37+0000] INFO - prefect | Preparing pipeline script
[2022-02-18 13:43:38+0000] INFO - prefect | Finished with status COMPLETED


In [ ]:
acdp_task = hapaseg.Hapaseg_run_acdp(
    inputs = {
        "coverage_dp_object":cov_dp_results["cov_dp_object"],
        "acdp_df":gen_acdp_results["acdp_df_pickle"],
        "num_samples":10,
        "cytoband_df":
    }